In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

train_data = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NueralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NueralNetwork().to(device)
print(model)

Using cuda device
NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.7369,  0.0867, -0.7225,  0.5346, -0.4034,  0.2919,  0.3003,  0.2116,
          0.3565, -0.6038,  0.0561, -0.1529, -0.2985, -0.2689, -0.9030, -0.8514,
          0.4444,  0.0073,  0.0592,  0.1422],
        [-0.2748, -0.0918, -0.1514,  0.3555, -0.6374,  0.2100,  0.4921,  0.2460,
          0.0801, -0.5021, -0.4492, -0.3592, -0.1301, -0.2755, -0.7681, -0.6855,
          0.5444,  0.1550,  0.0027,  0.3317],
        [-0.3262,  0.1404, -0.1219,  0.4002, -0.2356, -0.2290,  0.1125,  0.6478,
          0.2102, -0.6219, -0.4135, -0.3831, -0.1525, -0.4030, -0.8811, -0.5749,
          0.5803,  0.0888, -0.0239,  0.2007]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0867, 0.0000, 0.5346, 0.0000, 0.2919, 0.3003, 0.2116, 0.3565,
         0.0000, 0.0561, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4444, 0.0073,
         0.0592, 0.1422],
        [0.0000, 0.0000, 0.0000, 0.3555, 0.0000, 0.2100, 0.4921, 0.2460, 0.0801,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0164,  0.0011, -0.0208,  ...,  0.0316, -0.0180, -0.0114],
        [-0.0159, -0.0158,  0.0012,  ..., -0.0148,  0.0074,  0.0164]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0133,  0.0194], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0316, -0.0110,  0.0175,  ...,  0.0431, -0.0169, -0.0116],
        [-0.0413,  0.0426,  0.0400,  ..., -0.0251, -0.0336, -0.0231]],
       device='cuda:0', grad_fn=<Slice